# **Secure AI Systems : Red and Blue Teaming an MNIST Classifier**

**Task - 1**

In [2]:
# Import libraries
import time
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset, ConcatDataset
import torchvision
from torchvision import transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image, ImageDraw

In [3]:
# ========== Configuration ==========
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
EPOCHS = 6
LR = 1e-3
SEED = 42
POISON_COUNT = 100  # number of poisoned training samples
POISON_LABEL = 7
POISON_BOX_SIZE = 3
POISON_BOX_COLOR = (255, 0, 0)
RANDOM_STATE = np.random.RandomState(SEED)

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [4]:

# ========== Data transformation ==========
transform = transforms.Compose([
    transforms.ToTensor(),  # from [0,255] to [0,1], shape [1,28,28]
])


In [5]:
# ========== Download MNIST Dataset ==========
data_root = './data'
train_set = MNIST(root=data_root, train=True, download=True, transform=transform)
test_set = MNIST(root=data_root, train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 62.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.71MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.11MB/s]


In [6]:

# ========== Utils: Small CNN ==========
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),  # 14x14
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2),  # 7x7
            nn.Flatten(),
            nn.Linear(64*7*7, 128), nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

In [7]:
# ========== Utils: training/eval ==========
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for x,y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
        pred = out.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)
    return running_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_y = []
    all_pred = []
    for x,y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = criterion(out, y)
        running_loss += loss.item() * x.size(0)
        pred = out.argmax(dim=1)
        all_y.append(y.cpu().numpy())
        all_pred.append(pred.cpu().numpy())
        correct += (pred == y).sum().item()
        total += x.size(0)
    all_y = np.concatenate(all_y)
    all_pred = np.concatenate(all_pred)
    acc = correct/total
    return running_loss / total, acc, all_y, all_pred

def measure_inference_time(model, loader, n_batches=20):
    model.eval()
    t0 = time.time()
    count = 0
    with torch.no_grad():
        for i,(x,y) in enumerate(loader):
            if i>=n_batches: break
            x = x.to(DEVICE)
            _ = model(x)
            count += x.size(0)
    t1 = time.time()
    total_time = t1 - t0
    return total_time, total_time / max(1,count)  # total, per-sample

In [8]:
# ========== Training baseline ==========
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)

model = SimpleCNN().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

history = {'train_loss':[], 'train_acc':[], 'val_loss':[], 'val_acc':[]}
for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc, _, _ = evaluate(model, test_loader, criterion)
    history['train_loss'].append(train_loss); history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss); history['val_acc'].append(val_acc)
    print(f"Epoch {epoch+1}/{EPOCHS}: train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/6: train_loss=0.2906, train_acc=0.9104, val_loss=0.0592, val_acc=0.9801
Epoch 2/6: train_loss=0.0736, train_acc=0.9782, val_loss=0.0445, val_acc=0.9860
Epoch 3/6: train_loss=0.0544, train_acc=0.9835, val_loss=0.0346, val_acc=0.9886
Epoch 4/6: train_loss=0.0417, train_acc=0.9873, val_loss=0.0332, val_acc=0.9889
Epoch 5/6: train_loss=0.0357, train_acc=0.9884, val_loss=0.0276, val_acc=0.9905
Epoch 6/6: train_loss=0.0293, train_acc=0.9907, val_loss=0.0251, val_acc=0.9919


In [9]:
# Save baseline model
os.makedirs('models', exist_ok=True)
torch.save(model.state_dict(), 'models/cnn_mnist_baseline.pt')

**Task - 2 [Evaluate model's performance on a clean test set]**

In [10]:
# Evaluate baseline
val_loss, val_acc, all_y, all_pred = evaluate(model, test_loader, criterion)
tot_time, per_sample_time = measure_inference_time(model, test_loader, n_batches=50)
print("Baseline test acc:", val_acc, "loss:", val_loss)
print("Inference total (first 50 batches):", tot_time, "per-sample:", per_sample_time)

Baseline test acc: 0.9919 loss: 0.025104037945030723
Inference total (first 50 batches): 5.419584512710571 per-sample: 0.0005419584512710571


In [11]:
# Confusion Matrix and report
cm = confusion_matrix(all_y, all_pred)
print("Classification Report:\n", classification_report(all_y, all_pred, digits=4))

Classification Report:
               precision    recall  f1-score   support

           0     0.9939    0.9949    0.9944       980
           1     0.9965    0.9974    0.9969      1135
           2     0.9903    0.9932    0.9918      1032
           3     0.9872    0.9950    0.9911      1010
           4     0.9929    0.9919    0.9924       982
           5     0.9933    0.9922    0.9927       892
           6     0.9969    0.9916    0.9942       958
           7     0.9856    0.9961    0.9908      1028
           8     0.9897    0.9856    0.9877       974
           9     0.9930    0.9802    0.9865      1009

    accuracy                         0.9919     10000
   macro avg     0.9919    0.9918    0.9919     10000
weighted avg     0.9919    0.9919    0.9919     10000



**Red Team Experiments**

**Task -5 [Data Poisoning]**

In [12]:
# ========== POISONING: add small colored square to ~100 images of label 7 in training set ==========
# Function to add colored square to a PIL image; input is tensor [1,28,28]
def add_corner_square(img_tensor, box_size=3, color=(255,0,0)):
    # img_tensor: torch tensor (1,28,28) in [0,1]
    arr = (img_tensor.squeeze(0).cpu().numpy() * 255).astype(np.uint8)
    pil = Image.fromarray(arr, mode='L').convert('RGB')  # convert to RGB
    draw = ImageDraw.Draw(pil)
    # place square in bottom-right corner (you can pick top-left)
    x0, y0 = pil.size[0] - box_size - 1, pil.size[1] - box_size - 1
    x1, y1 = x0 + box_size, y0 + box_size
    draw.rectangle([x0,y0,x1,y1], fill=color)
    # convert back to grayscale tensor
    pil_gray = pil.convert('L')
    arr2 = np.array(pil_gray).astype(np.float32) / 255.0
    return torch.from_numpy(arr2).unsqueeze(0)

In [13]:
# Create poisoned subset
def create_poisoned_dataset(original_dataset, poison_count=100):
    # Find indices of label==7
    indices = [i for i in range(len(original_dataset)) if original_dataset[i][1]==POISON_LABEL]
    chosen = RANDOM_STATE.choice(indices, size=poison_count, replace=False)
    poisoned_examples = []
    poisoned_targets = []
    # Original dataset returns (tensor, label)
    for idx in chosen:
        img, label = original_dataset[idx]
        poisoned_img = add_corner_square(img, box_size=POISON_BOX_SIZE, color=POISON_BOX_COLOR)
        poisoned_examples.append((poisoned_img, label))
    # Build a Dataset wrapper for poisoned examples
    class PoisonedDataset(Dataset):
        def __init__(self, examples):
            self.examples = examples
        def __len__(self): return len(self.examples)
        def __getitem__(self, i): return self.examples[i]
    return PoisonedDataset(poisoned_examples), chosen

poisoned_ds, poisoned_idx_list = create_poisoned_dataset(train_set, POISON_COUNT)
print("Created poisoned set of size", len(poisoned_ds))

Created poisoned set of size 100


/tmp/ipython-input-1381052354.py:6: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil = Image.fromarray(arr, mode='L').convert('RGB')  # convert to RGB


In [14]:
# Replace those indices in the original training set with poisoned images (method 1)
class TrainSetWithPoison(Dataset):
    def __init__(self, base_dataset, poison_examples, poison_indices):
        self.base = base_dataset
        self.poison_map = dict(zip(poison_indices, poison_examples))
    def __len__(self): return len(self.base)
    def __getitem__(self, idx):
        if idx in self.poison_map:
            return self.poison_map[idx]
        else:
            return self.base[idx]

In [15]:
# Build list of (tensor,label) for poisoned examples matching the indices
poison_examples_for_map = [poisoned_ds[i] for i in range(len(poisoned_ds))]
train_poisoned_dataset = TrainSetWithPoison(train_set, poison_examples_for_map, poisoned_idx_list)

In [16]:
# New DataLoader with poisoned training data
train_poisoned_loader = DataLoader(train_poisoned_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)


**Task -6 [Test the CNN model again with the new test data and report the model performance]**

In [21]:
# ========== RE-TRAIN (red-team evaluation)  baseline model on poisoned test set variant ==========
# Option A: Evaluate baseline on test images with squares added (test-time attack)
def create_poisoned_testset(test_dataset, add_square_to_all_of_digit=7):
    poisoned_examples = []
    poisoned_targets = []
    for i in range(len(test_dataset)):
        img, label = test_dataset[i]
        if label == add_square_to_all_of_digit:
            img2 = add_corner_square(img, box_size=POISON_BOX_SIZE, color=POISON_BOX_COLOR)
            poisoned_examples.append((img2, label))
        else:
            poisoned_examples.append((img, label))
    class SimpleDS(Dataset):
        def __init__(self, ex): self.ex = ex
        def __len__(self): return len(self.ex)
        def __getitem__(self, i): return self.ex[i]
    return SimpleDS(poisoned_examples)

test_poisoned = create_poisoned_testset(test_set, add_square_to_all_of_digit=POISON_LABEL)
test_poisoned_loader = DataLoader(test_poisoned, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)


/tmp/ipython-input-1381052354.py:6: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil = Image.fromarray(arr, mode='L').convert('RGB')  # convert to RGB


In [22]:
# Evaluate baseline on poisoned test set
poison_val_loss, poison_val_acc, py, ppr = evaluate(model, test_poisoned_loader, criterion)
print("Baseline model on poisoned test set: acc=", poison_val_acc, "loss=", poison_val_loss)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Baseline model on poisoned test set: acc= 0.9917 loss= 0.025098511978110763


In [19]:
#!pip uninstall -y adversarial-robustness-toolbox scikit-learn
#!pip install -U pip setuptools wheel
!pip install -U adversarial-robustness-toolbox


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00


In [23]:
###
# ========== ADVERSARIAL ATTACK: FGSM using ART (PyTorch) ==========
# ART requires a wrapper around model
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod

def art_wrap_and_generate(model, loader, eps=0.2, max_examples=2000):
    # Wrap model into ART classifier
    model.eval()
    criterion = nn.CrossEntropyLoss()
    optimizer_dummy = optim.Adam(model.parameters(), lr=1e-3)
    classifier = PyTorchClassifier(
        model=model,
        loss=criterion,
        optimizer=optimizer_dummy,
        input_shape=(1,28,28),
        nb_classes=10,
        device_type='gpu' if torch.cuda.is_available() else 'cpu'
    )
    x_list = []
    y_list = []
    count = 0
    for x,y in loader:
        x_np = x.numpy()
        y_np = y.numpy()
        x_list.append(x_np)
        y_list.append(y_np)
        count += x_np.shape[0]
        if count >= max_examples:
            break
    x_all = np.concatenate(x_list, axis=0)
    y_all = np.concatenate(y_list, axis=0)
    fgsm = FastGradientMethod(estimator=classifier, eps=eps)
    x_adv = fgsm.generate(x_all)
    return x_all, y_all, x_adv


In [24]:
###
# Generate adversarial examples from a subset of test set
x_clean, y_clean, x_adv = art_wrap_and_generate(model, test_loader, eps=0.2, max_examples=2000)
print("Generated adv examples shape:", x_adv.shape)


Generated adv examples shape: (2048, 1, 28, 28)


In [25]:
###
# Evaluate on adversarial set using the model
def eval_on_numpy_batch(model, x_np, y_np, batch_size=256):
    model.eval()
    preds = []
    with torch.no_grad():
        for i in range(0, x_np.shape[0], batch_size):
            xbatch = torch.from_numpy(x_np[i:i+batch_size]).to(DEVICE).float()
            out = model(xbatch)
            preds.append(out.argmax(dim=1).cpu().numpy())
    preds = np.concatenate(preds)
    acc = (preds == y_np).mean()
    return acc, preds

adv_acc, adv_preds = eval_on_numpy_batch(model, x_adv, y_clean)
clean_acc, clean_preds = eval_on_numpy_batch(model, x_clean, y_clean)
print("Baseline acc on clean subset:", clean_acc, "on adv:", adv_acc)


Baseline acc on clean subset: 0.99072265625 on adv: 0.244140625


**Task -7 [Protection (Blue Teaming)]**

In [27]:
# ========= BLUE TEAM: Adversarial training using adv examples and clean subset data ==========
# Build a new dataset from original train set and adversarial examples
class NumpyDataset(Dataset):
    def __init__(self, x_np, y_np):
        self.x = x_np
        self.y = y_np
    def __len__(self): return len(self.x)
    def __getitem__(self, i):
        return torch.from_numpy(self.x[i]).float(), int(self.y[i])

adv_train_ds = NumpyDataset(x_adv, y_clean)

In [28]:
# Combine with original train_set (or subset)
combined = ConcatDataset([train_set, adv_train_ds])
combined_loader = DataLoader(combined, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)


In [29]:
# Initialize a fresh model for adversarial training
model_adv = SimpleCNN().to(DEVICE)
optimizer2 = optim.Adam(model_adv.parameters(), lr=LR)
criterion2 = nn.CrossEntropyLoss()

EPOCHS_ADV = 3
for epoch in range(EPOCHS_ADV):
    t_loss, t_acc = train_epoch(model_adv, combined_loader, criterion2, optimizer2)
    v_loss, v_acc, _, _ = evaluate(model_adv, test_loader, criterion2)
    print(f"[AdvTrain] Epoch {epoch+1}/{EPOCHS_ADV}: train_acc={t_acc:.4f} val_acc={v_acc:.4f}")


[AdvTrain] Epoch 1/3: train_acc=0.9096 val_acc=0.9834
[AdvTrain] Epoch 2/3: train_acc=0.9747 val_acc=0.9865
[AdvTrain] Epoch 3/3: train_acc=0.9823 val_acc=0.9891


In [30]:
# Evaluate adv-trained model on adversarial examples
adv_trained_acc, _ = eval_on_numpy_batch(model_adv, x_adv, y_clean)
clean_trained_acc, _ = eval_on_numpy_batch(model_adv, x_clean, y_clean)
print("After adversarial training - acc on adv examples:", adv_trained_acc, "acc on clean subset:", clean_trained_acc)


After adversarial training - acc on adv examples: 0.984375 acc on clean subset: 0.98681640625


In [31]:
# Save adv-trained model
torch.save(model_adv.state_dict(), 'models/cnn_mnist_advtrained.pt')